In [ ]:
import scipy.special
import numpy as np
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from time import time
import numqi
from matplotlib.ticker import MaxNLocator
import matplotlib.colors as mcolors
from utils import GeometricCoherenceModel, get_real_equal_prob_state_geometric_coherence

In [ ]:
def noise_maximally_coherent_state(dim, p):
    tmp0 = np.ones(dim, dtype=np.float64)/np.sqrt(dim)
    maximally_coherent_dm = tmp0.reshape(-1,1) * tmp0.conj()
    noise_dm = p*maximally_coherent_dm + (1-p)*np.eye(dim)/dim
    return noise_dm

In [ ]:
dim = 30
p_list = np.linspace(0, 1, 20)

model = GeometricCoherenceModel(dim, num_term=4*dim, temperature=0.3)
kwargs = dict(theta0='uniform', num_repeat=3, tol=1e-14, print_every_round=0)

gc_list = []

for p in tqdm(p_list):
    model.set_density_matrix(noise_maximally_coherent_state(dim, p))
    theta_opt = numqi.optimize.minimize(model, **kwargs).fun
    with torch.no_grad():
        gc_list.append(model(use_temperature=False).item())
gc_list = np.array(gc_list)

gc_analytical = get_real_equal_prob_state_geometric_coherence(dim, p_list)


In [ ]:
err_list = np.abs(gc_list-gc_analytical)
print(f'max error: {np.max(err_list)}')

fig,ax = plt.subplots()
ax.plot(p_list, gc_list, label='numerical')
ax.plot(p_list, gc_analytical, 'x', label='analytical')
ax.legend()
ax.set_xlabel('p')
ax.set_ylabel('geometric coherence')
ax.set_title(f'dim={dim}')
fig.tight_layout()

In [ ]:
kwargs = dict(theta0='uniform', num_repeat=3, tol=1e-14, print_every_round=0)
data_list = []
p_list = np.linspace(0, 1, 20)
for dim in tqdm(range(2, 51)):
    model = GeometricCoherenceModel(dim, num_term=2*dim, temperature=0.3)
    err_list = []
    t_list = []
    for p in p_list:
        target_dm = noise_maximally_coherent_state(dim, p)
        model.set_density_matrix(target_dm)
        t0 = time()
        theta_opt = numqi.optimize.minimize(model, **kwargs).fun
        t_list.append(time()-t0)
        with torch.no_grad():
            err_list.append(np.abs(model(use_temperature=False).item()-get_real_equal_prob_state_geometric_coherence(dim, p)))
    data_list.append([dim, np.max(err_list), np.mean(t_list)])
data_np = np.array(data_list)

In [ ]:
dim_list = data_np[:,0]
max_err_list = data_np[:,1]
mean_t_list = data_np[:,2]

# choose colors from tableau

fig, ax1 = plt.subplots()
ax1.plot(dim_list, max_err_list, mcolors.TABLEAU_COLORS['tab:blue'])
ax1.set_xlabel(r'Dimension $d$')
ax1.xaxis.set_major_locator(MaxNLocator(integer=True))
ax1.set_ylabel('Max abs error', color=mcolors.TABLEAU_COLORS['tab:blue'])
ax1.set_yscale('log')
ax1.tick_params(axis='y', labelcolor=mcolors.TABLEAU_COLORS['tab:blue'])

ax2 = ax1.twinx()
ax2.plot(dim_list, mean_t_list, mcolors.TABLEAU_COLORS['tab:orange'])
ax2.set_ylabel('Time (s)', color=mcolors.TABLEAU_COLORS['tab:orange'])
ax2.tick_params(axis='y', labelcolor=mcolors.TABLEAU_COLORS['tab:orange'])
     
fig.tight_layout()
fig.savefig('noise_maximally_coherent_state.pdf')
